So this is the lab...
Logistic regression and stuff
so yeah... we need paramters for regularization
none, L1, L2, this can jsut be a flag with hyperparamters and overloaded functions for the regulairzation...
well maybe not overloading but a method that we call that calls other methods called L1 and L2
we can compare different regularizations within the same method, that way its legit. 
for the methods... Were suggetsted to the Stochiastic graident descent. mini-batch gradient descent for our steepest descent and the hessian for the quasi newton thingy. Frick do we gotta write our own hessian? Thatll suck big time

Step1.
Read data and encode locations as one hot encoded
Step2.
Split into 80% train and 20 %test after merging.
Step3. Write regularization
Do these tonight


We did both of the regularzations but not together \for reg we do ridge lasso or elastic net 
gradient[1:] = regularization + get gradient

When trying to shop for clothing items online, it can be difficult to determine the correct sizing for your shape. Most website just have your basic options of small, medium, large, etc., but how do you know which option would work best for you? Life would be so much easier if you knew the item you ordered would come in fitting exactly the way you imagined it would. From using the content in this dataset, we can write an algorithm that when given specific measurements, it will map to previous successful similar entries and return the best fit for your features. The ability to achieve this approximate "fit", allows the customer to feel more confident in their sizing selection process.

According to https://www.shopify.com/enterprise/ecommerce-returns, seventy-two percent of all of their customer returns are due to issues with the fit, sizing, style, etc. When researching different companies on their return rates, the customer-preference based returns are constantly receiving the highest percentages.

With a successful classification algorithm, this could be deployed into hundreds of online shopping stores and provide a better experience throughout. For example, people would not need to go through the hassle of repackaging and shipping their item, the money they are having to spend for this process, working with customer service on an exchange or refund, etc. On the business side, companies will be able to help more customers due to the decrease of services needed, receive higher ratings from customer satisfaction, and much more. By having this algorithm in place, the world of online shopping would increase, and it would be easier to obtain the products you need, when you do not have time in your schedule, and have no doubts about the fit of your product.

[Dataset Source](https://www.kaggle.com/rmisra/clothing-fit-dataset-for-size-recommendation/data)


In [1]:
#this is the block that simply reads the data
import os
import re
import math
import numpy as np
import pandas as pd
def translate_height(height):
    height = str(height)
    if height == "nan":
        return None
    l = re.findall(r'[0-9]+',height)
    if len(l) == 1:
        return int(l[0])*12
    else:
        return int(l[0])*12 + int(l[1])
df = []
y = []
df = pd.read_json('clothing-fit-dataset-for-size-recommendation/modcloth_final_data.json', lines=True)
di = {"very short": 1, "slightly short": 2, "just right": 3, "slightly long": 4, "very long": 5}
df = df.replace({"length": di})
di = {"small": 1, "fit": 2, "large": 3}
df = df.replace({"fit": di})
df["height"] = df["height"].map(lambda x: translate_height(x))

di = {"aa": 0, "a": 1, "b": 2, "c": 3, "d": 4, "dd/e": 5, "ddd/f": 6, "dddd/g": 7, "h": 8, "i": 9, "j": 10, "k": 11}
df = df.replace({"cup size": di})
#maybe add shoe width back?
y = df['fit']
#maybe add length back?
df = df.drop(columns=["item_id", "category", "user_name", "user_id", "shoe width", "review_summary", "review_text", "fit"])
print("NaNs per column, if more than 20% we will drop otherwise")
for column in df:
    print(df[column].isna().sum(), column)
    df[column] = pd.to_numeric(df[column], errors="coerce")
df = df.drop(columns=["waist", "bust", "shoe size", "length", "hips"])
df = df.fillna(df.mean())
print(df.head())
#big issue here is predicting columns with shoes? wtf there are so many NaNs. Not sure what to do about shoe products?
x = df.values.astype(int)
x

NaNs per column, if more than 20% we will drop otherwise
6018 bra size
70936 bust
6255 cup size
1107 height
26726 hips
35 length
68 quality
54875 shoe size
0 size
79908 waist
    bra size  cup size    height  quality  size
0  34.000000       4.0  66.00000      5.0     7
1  36.000000       2.0  62.00000      3.0    13
2  32.000000       2.0  67.00000      2.0     7
3  35.972125       5.0  65.14642      5.0    21
4  36.000000       2.0  62.00000      5.0    18


array([[34,  4, 66,  5,  7],
       [36,  2, 62,  3, 13],
       [32,  2, 67,  2,  7],
       ...,
       [32,  7, 64,  5, 12],
       [35,  3, 63,  4, 12],
       [32,  4, 72,  4,  4]])

In [2]:
#this is for the renttherunway stuff
df = pd.read_json('clothing-fit-dataset-for-size-recommendation/renttherunway_final_data.json', lines=True)
df.head()

,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,review_summary,review_text,size,user_id,weight
0,28.0,hourglass,34d,romper,fit,"5' 8""",2260466,10.0,vacation,"April 20, 2016",So many compliments!,An adorable romper! Belt and zipper were a lit...,14,420272,137lbs
1,36.0,straight & narrow,34b,gown,fit,"5' 6""",153475,10.0,other,"June 18, 2013",I felt so glamourous!!!,I rented this dress for a photo shoot. The the...,12,273551,132lbs
2,116.0,NaN,NaN,sheath,fit,"5' 4""",1063761,10.0,party,"December 14, 2015",It was a great time to celebrate the (almost) ...,This hugged in all the right places! It was a ...,4,360448,NaN
3,34.0,pear,34c,dress,fit,"5' 5""",126335,8.0,formal affair,"February 12, 2014",Dress arrived on time and in perfect condition.,I rented this for my company's black tie award...,8,909926,135lbs
4,27.0,athletic,34b,gown,fit,"5' 9""",616682,10.0,wedding,"September 26, 2016",Was in love with this dress !!!,I have always been petite in my upper body and...,12,151944,145lbs


In [3]:
df["cup size"] = df["bust size"].map(lambda x:  re.findall(r'[A-Za-z]+',str(x))[0] )
df["bust size"] = df["bust size"].map(lambda x: re.findall(r'[0-9]+',str(x)) ).map(lambda x:  np.nan if len(x) == 0 else x[0])
di = {"aa": 0, "a": 1, "b": 2, "c": 3, "d": 4, "dd": 5, "ddd": 6, "f": 6, "g": 7, "h": 8, "i": 9, "j": 10, "k": 11}
df = df.replace({"cup size": di})
di = {"small": 1, "fit": 2, "large": 3}
di = {"small": 1, "fit": 2, "large": 3}
df = df.replace({"fit": di})
y1 = df["fit"]
df = df.drop(columns=['fit', 'age', 'body type', 'category', 'item_id', 'rented for', 'review_date', 'review_summary', 'review_text', 'user_id', 'weight'])
df["height"] = df["height"].map(lambda x: translate_height(x))
#df = df[['bust size', 'cup size', 'height', 'rating', 'size']]

for col in df:
    print(df[col].isna().sum(), column)
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[col] = df[col].fillna(value=df[col].mean())
print(df.head())
x1 = df.values.astype(int)
np.append(x, x1)
np.append(y, y1)

18411 waist
677 waist
82 waist
0 waist
0 waist
   bust size  height  rating  size  cup size
0  34.000000    68.0    10.0    14  4.000000
1  34.000000    66.0    10.0    12  2.000000
2  34.201271    64.0    10.0     4  2.958239
3  34.000000    65.0     8.0     8  3.000000
4  34.000000    69.0    10.0    12  2.000000


array([1, 1, 1, ..., 2, 2, 2])

In [4]:
#now split into training and testing data
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = .80)
y_train

72837    2
54733    1
72873    2
24557    2
30416    3
12668    2
22895    1
77163    2
81356    1
77080    1
26279    2
236      3
24646    2
67887    2
64166    2
36991    2
28810    2
42834    2
32736    2
4997     2
44177    3
27960    2
79140    2
51697    2
69482    3
14617    2
26497    2
13454    1
22603    2
82478    2
        ..
59202    2
45106    2
66974    2
66429    2
56567    2
71224    2
35515    2
22505    2
41428    2
14141    2
31797    1
68170    2
33776    3
73866    1
50920    3
77038    1
28194    3
63726    2
50436    1
51874    1
53657    3
24764    2
17723    2
76482    3
49690    1
26673    2
47007    2
73553    2
72733    2
81196    2
Name: fit, Length: 16558, dtype: int64

In [5]:
#test logistic regresion to see if my splits are working
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
clf = LogisticRegression(fit_intercept=False)
clf.fit(X_train, y_train)

yhat = clf.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

Accuracy of:  0.6845784515038048


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


## Regurlzarization


In [6]:
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # this typically gives us the probability of a specific class
        # we need a bias term 
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        # this will take the x with the bias term and multiplies it by w. This will give us a scalr quantity
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        # if we actually want to do a prediction we are seeing if it is greater than point 5. vector of ones were it is greater than .5 and 0 when it is below .5
        # When one is greater than the other it would be greater than .5
        return (self.predict_proba(X)>0.5) #return the actual prediction

In [7]:
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        #for each vector x we want to out it throught he sigmoid suntract it from y and add it to the gradient function
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_bias=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        #return gradient divided by the total number of points
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        #this is going to get features and labels. We are going to say x wtih a bias is ther (col of 1s)
        #the shape is the # of samples (rows ) by # of features (cols)
        #w always starts at 0 usually. 
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        # whats the gradient?? and internall we are updating self.w_ in this
        
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            
            self.w_ += gradient*self.eta # multiply by learning rate 

           

In [8]:
from scipy.special import expit
class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)

In [9]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
#         for i, val in self.unique_[:-1]:
#             y_binary = y.map(lambda q: q == val or q == self.unique[i+1])
#             blr = VectorBinaryLogisticRegression(self.eta,self.iters)
#             blr.fit(X,y_binary)
#             # add the trained classifier to the list
#             self.classifiers_.append(blr)
        for i,yval in enumerate(self.unique_): # for each unique value
           y_binary = (y==yval) # create a binary problem whenever class is 0 (big class), y binary will be 1
           # train the binary classifier for this class
           blr = VectorBinaryLogisticRegression(self.eta,self.iters)
           blr.fit(X,y_binary)
           # add the trained classifier to the list
           self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row
    
#print(X)
#print(y)

In [10]:
#yhat = lr.predict(X_test)
#print('Accuracy of: ',accuracy_score(y_test,yhat))


In [11]:
class RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
    # extend init functions
    def __init__(self, C=0.0,reg="BOTH", **kwds):        
        # need to add to the original initializer 
        self.C = C
        self.reg = reg
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
        
    # extend previous class to change functionality
    def _get_gradient(self,X,y):
        # call get gradient from previous class
        gradient = super()._get_gradient(X,y)
        # add in regularization (to all except bias term)
        #this is L1
        div = np.divide(np.absolute(self.w_[1:]).T[0],
                        self.w_[1:,0],
                        out=np.zeros_like(np.absolute(self.w_[1:]).T[0]),
                        where=self.w_[1:,0]!=0)
        div = div.T.reshape(self.w_[1:].shape)
        if self.reg == "L1" or self.reg == "BOTH":
            gradient[1:] += -1 * div  * self.C
        if self.reg == "BOTH" or self.reg == "L2":
            gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
        

In [12]:
# now redefine the Logistic Regression Function where needed
import itertools
def get_uniq(s, s1, s2):
    if s == s1 or s == s2:
        return s
    else:
        return None
class RegularizedLogisticRegression(LogisticRegression):
    def __init__(self, C=0.0,reg="BOTH", **kwds):        
        # need to add to the original initializer 
        self.C = C
        self.reg = reg
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        iters = list(itertools.combinations(self.unique_, 2))
        print(iters, "ITER")
#         for i in iters:
            
#             y_binary = y.map(lambda q: get_uniq(q, i[0], i[1]) )
#             x_uniq = np.where(np.isin(y, i))
#             print(y_binary.dropna(), len(y_binary.dropna()))
            
#             blr = RegularizedBinaryLogisticRegression(eta=self.eta,
#                                                       iterations=self.iters,
#                                                       C=self.C,
#                                                      reg=self.reg)
#             #print(len(X[x_uniq[0],:]),len(y_binary.dropna()))
#             blr.fit(X[x_uniq[0],:],y_binary.dropna())
#             # add the trained classifier to the list
#             #print(blr)
#             self.classifiers_.append(blr)
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                      C=self.C)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

In [13]:
lr = RegularizedLogisticRegression(eta=0.1,
                                           iterations=2500,
                                           C=.4, 
                                  reg = "L2") # get object(0.1,500)
lr.fit(X_train,y_train)
print(lr)


[(1, 2), (1, 3), (2, 3)] ITER
MultiClass Logistic Regression Object with coefficients:
[[-7.91562457e+00 -1.26049310e+00 -3.26425859e-02 -2.57153874e+00
  -1.04031988e-01 -2.41319165e-01]
 [ 5.72891050e+00  6.70700128e-01  3.38867415e-02  1.55558051e+00
   4.61711574e-02 -8.16635860e-02]
 [-7.39421026e+00  1.08491126e-01  6.03669360e-03  1.53079341e-01
   7.99092227e-03  9.20618737e-02]]


In [14]:
yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))
i, l = np.unique(yhat, return_counts=True)
i

Accuracy of:  0.684563353061964


array([2])

In [15]:
#minibatch s essentially just running regression a smaller batch. This will be much faster with hopefully similar accuracy.
from numpy import random
class StochasticGradientDescent(RegularizedLogisticRegression):
    def __init__(self,size=1000,reg="BOTH", **kwds):
        self.reg = reg
        self.size = size
            #make sure to pass c, eta and iterations
        super().__init__(**kwds) # call parent initializer
        
    def fit(self, X, y):
        num_samples, num_features = X.shape
        #get a random sample 
        x_slice = X[self.size:self.size*2,:]
        y_slice = y[self.size:self.size*2]
        #print(y_slice)
        self.unique_ = np.unique(y_slice) # get each unique class value
        
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        #then we will
        #print(np.unique(y_slice))
        #there are nans in here
        #print(self.unique_)
        #for i, val in enumerate(self.unique_[:-1]):
            #y_binary = y.map(lambda q: get_uniq(q, val, self.unique_[i+1]) )
            #x_uniq = np.where(np.isin(y, [val, self.unique_[i+1]]))
            #print(len(X), len(y_binary.dropna()), "LENGTHS")
            #print((np.where(np.isin(y, [val, self.unique_[i+1]]))[0]), "unique len")
            #print((x_uniq[0]),len(x_uniq[0]), "UNIQUIQ")
            #print(X[x_uniq[0],:], "uniq")
            #blr = RegularizedBinaryLogisticRegression(eta=self.eta,
            #                                          iterations=self.iters,
            #                                          C=self.C,
            #                                         reg=self.reg)
            #print(len(X[x_uniq[0],:]),len(y_binary.dropna()))
            #blr.fit(X[x_uniq[0],:],y_binary.dropna())
            # add the trained classifier to the list
            #print(blr)
            #self.classifiers_.append(blr)
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y_slice==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                          C=self.C,
                                                     reg=self.reg)
            blr.fit(x_slice,y_binary)
                # add the trained classifier to the list
            self.classifiers_.append(blr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        #stochastic takes a single sample 



In [16]:
#print(y_train)
#5194
ls = StochasticGradientDescent(eta=0.1,
                                           iterations=2500, size=3000,
                                           C=1,
                              reg="L2") # get object(0.1,500)

#print( len(y_train))

ls.fit(X_train,y_train)
print(ls)


MultiClass Logistic Regression Object with coefficients:
[[-1.73458853e+01 -2.90708755e+00 -3.63888771e-01 -5.30915788e+00
  -4.36688416e-01 -8.93266861e-01]
 [ 5.35000002e+00  5.65109290e-02 -7.49701188e-02  3.02262295e-01
  -1.07788620e-02 -2.37622951e-02]
 [-1.72260522e+01 -2.93168505e+00 -3.76098089e-01 -5.36141280e+00
  -4.16719555e-01 -8.94898148e-01]]


In [17]:
yhat = ls.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))
i, l = np.unique(yhat, return_counts=True)
i

Accuracy of:  0.684563353061964


array([2])

In [21]:
class Newton(RegularizedLogisticRegression):
    def sig(theta):
        return 1/(1+np.exp(-theta)) 
    
    def __init__(self,size=1000, **kwds):
        self.size = size
            #make sure to pass c, eta and iterations
        super().__init__(**kwds) # call parent initializer
        
    # extend previous class to change functionality
    def _get_gradient(self,X,y):
        # call get gradient from previous class
        gradient = super()._get_gradient(X,y)
        
        S = diag(self.sig(X @ self.w_)*(1-self.sig(X @ self.w_)))
        hessian = (X.T*S*X)
        H_inv = np.linalg.inv(hessian)
#         g = (X.T((self.sig(X @ self.w_)) - y))
        newt_update = H_inv * gradient
        gradient = newt_update
        return gradient
        
    def fit(self, X, y):   
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                      C=self.C)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            print(blr)
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T


In [22]:
ls = Newton(eta=0.1,iterations=2500, 
                   size=1000, C=0.4, reg="L2")
ls.fit(X_train,y_train)
yhat = ls.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))
    

Binary Logistic Regression Object with coefficients:
[[-7.91562457]
 [-1.2604931 ]
 [-0.03264259]
 [-2.57153874]
 [-0.10403199]
 [-0.24131917]]
Binary Logistic Regression Object with coefficients:
[[ 5.7289105 ]
 [ 0.67070013]
 [ 0.03388674]
 [ 1.55558051]
 [ 0.04617116]
 [-0.08166359]]
Binary Logistic Regression Object with coefficients:
[[-7.39421026e+00]
 [ 1.08491126e-01]
 [ 6.03669360e-03]
 [ 1.53079341e-01]
 [ 7.99092227e-03]
 [ 9.20618737e-02]]
Accuracy of:  0.684563353061964


In [23]:
import time
from sklearn.metrics import confusion_matrix
lr_scores=[]
ls_scores = []
nt_scores = []
lr_confusion_matrices = {}
ls_confusion_matrices = {}
nt_confusion_matrices = {}
accuracy_scores = {}
#this block here will get the times of the blocks and save them and their accuracy
print(np.unique(y_train, return_counts=True))
for reg in ["L1", "L2", "BOTH"]:
    print(reg)
    for c in [.001, .01, .1, 3]:
        print(c, "C")
        for e in [.001, .01, .1]:
            print(e, "E")
            start = time.time()
            lr = RegularizedLogisticRegression(eta=e,
                                               iterations=2500,
                                               C=c, 
                                      reg = reg)
            lr.fit(X_train, y_train)
            yhat = lr.predict(X_test)
            end = time.time()
            lr_scores.append((end-start, accuracy_score(y_test, yhat)))
            #generate confusion
            print(np.unique(yhat, return_counts=True))
            print(accuracy_score(y_test, yhat))
            m = (confusion_matrix(y_test, yhat))
            accuracy_scores[(reg, c, e, "lr")] = accuracy_score(y_test, yhat)
            lr_confusion_matrices[(reg, c, e)] = m
            
            start = time.time()
            ls = StochasticGradientDescent(eta=e,iterations=2500, size=1000, C=c,reg=reg)
            ls.fit(X_train, y_train)
            yhat = ls.predict(X_test)
            print(np.unique(yhat, return_counts=True))
            print(accuracy_score(y_test, yhat))
            ls_confusion_matrices[(reg, c, e )] = confusion_matrix(y_test, yhat)
            accuracy_scores[(reg, c, e, "ls")] = accuracy_score(y_test, yhat)
            end = time.time()
            ls_scores.append((end-start, accuracy_score(y_test, yhat)))
            
            start = time.time()
            nt = Newton(eta=e,iterations=2500, C=c, reg = reg)
            nt.fit(X_train, y_train)
            yhat = nt.predict(X_test)
            end = time.time()
            nt_scores.append((end-start, accuracy_score(y_test, yhat)))
            
            #generate confusion
            nt_confusion_matrices[(reg, c, e )] = confusion_matrix(y_test, yhat)
            accuracy_scores[(reg, c, e, "nt")] = accuracy_score(y_test, yhat)
            nt_scores.append((end-start, accuracy_score(y_test, yhat)))
            
print(nt_scores)
            

(array([1, 2, 3]), array([ 2589, 11417,  2552]))
L1
0.001 C
0.001 E
[(1, 2), (1, 3), (2, 3)] ITER
(array([2]), array([66232]))
0.684563353061964
(array([1, 2]), array([   63, 66169]))
0.6846841405966905
Binary Logistic Regression Object with coefficients:
[[-0.00214301]
 [-0.00969978]
 [ 0.01764004]
 [-0.01636802]
 [-0.15345008]
 [ 0.02075314]]
Binary Logistic Regression Object with coefficients:
[[ 0.00152712]
 [ 0.01792046]
 [-0.00636518]
 [-0.00040998]
 [ 0.16381699]
 [-0.03419006]]
Binary Logistic Regression Object with coefficients:
[[-0.00109255]
 [-0.03363519]
 [-0.01052506]
 [-0.01288879]
 [-0.0302366 ]
 [ 0.03843873]]
0.01 E
[(1, 2), (1, 3), (2, 3)] ITER
(array([2]), array([66232]))
0.684563353061964
(array([1, 2]), array([ 1111, 65121]))
0.6820268148327092
Binary Logistic Regression Object with coefficients:
[[-0.01278986]
 [-0.11192646]
 [ 0.12045092]
 [-0.23950815]
 [-1.25894657]
 [-0.01648951]]
Binary Logistic Regression Object with coefficients:
[[ 0.00945262]
 [ 0.127503

(array([2]), array([66232]))
0.684563353061964
(array([1, 2]), array([45646, 20586]))
0.3683415871482063
Binary Logistic Regression Object with coefficients:
[[-0.1522988 ]
 [-1.53913604]
 [ 0.85682547]
 [-3.07421352]
 [-9.6231859 ]
 [-0.20893808]]
Binary Logistic Regression Object with coefficients:
[[ 0.05703793]
 [ 0.15023349]
 [-0.29188237]
 [-0.87243445]
 [12.28004937]
 [-5.07006923]]
Binary Logistic Regression Object with coefficients:
[[-0.06027382]
 [-3.17350192]
 [-0.77274114]
 [-3.74430422]
 [-1.83277095]
 [ 0.12531243]]
0.01 C
0.001 E
[(1, 2), (1, 3), (2, 3)] ITER
(array([2]), array([66232]))
0.684563353061964
(array([1, 2]), array([   28, 66204]))
0.6846690421548496
Binary Logistic Regression Object with coefficients:
[[-0.0026519 ]
 [-0.0037277 ]
 [ 0.00228917]
 [-0.01964066]
 [-0.13328671]
 [ 0.01918324]]
Binary Logistic Regression Object with coefficients:
[[ 2.06398554e-03]
 [ 1.36829271e-02]
 [-3.29666618e-06]
 [ 2.38130506e-03]
 [ 1.45071027e-01]
 [-3.27846711e-02]]
B

(array([2, 3]), array([61352,  4880]))
0.6621270684865322
(array([2, 3]), array([59641,  6591]))
0.6534152675443894
Binary Logistic Regression Object with coefficients:
[[-0.01576481]
 [-0.01142102]
 [ 0.0267442 ]
 [-0.09007371]
 [-0.9664444 ]
 [ 0.07594929]]
Binary Logistic Regression Object with coefficients:
[[ 0.01468636]
 [ 0.10226017]
 [-0.01169266]
 [ 0.13284121]
 [ 1.18821805]
 [-0.50081183]]
Binary Logistic Regression Object with coefficients:
[[-0.00452668]
 [-0.05906639]
 [ 0.00112211]
 [ 0.01236305]
 [-0.05190307]
 [ 0.18259878]]
0.1 E
[(1, 2), (1, 3), (2, 3)] ITER
(array([2]), array([66232]))
0.684563353061964
(array([2]), array([66232]))
0.684563353061964
Binary Logistic Regression Object with coefficients:
[[-0.32646504]
 [-0.3634087 ]
 [ 0.16613658]
 [-0.87690942]
 [-2.66527243]
 [ 0.22641567]]
Binary Logistic Regression Object with coefficients:
[[ 0.30801845]
 [ 2.52282466]
 [ 0.19865858]
 [ 3.96391629]
 [ 3.37411174]
 [-0.6135679 ]]
Binary Logistic Regression Object 

In [28]:
import plotly.graph_objects as go

nt_avg_time = 0
lr_avg_time = 0
ls_avg_time = 0
for i in range(len(nt_scores)):
    nt_avg_time += nt_scores[i][0]
nt_avg_time /= len(nt_scores)

for i in range(len(lr_scores)):
    lr_avg_time += lr_scores[i][0]
lr_avg_time /= len(lr_scores)

for i in range(len(ls_scores)):
    ls_avg_time += ls_scores[i][0]
ls_avg_time /= len(ls_scores)


methods=['Steepest Descent', 'Stochastic', 'Newton\'s Method']

fig = go.Figure([go.Bar(x=methods, y=[lr_avg_time, ls_avg_time, nt_avg_time])])
fig.update_layout(title_text='Average Run-time Per Method')
fig.show()


It appears that timewise, Stochastic Gradient Descent is the most efficient optimization technique, with Newton's Method and the Steepest Descent taking a similar amount of time.

In [29]:
for item, val in ls_confusion_matrices:
    print(val)

ValueError: too many values to unpack (expected 2)

In [30]:
#now we are going to plot it
#how????
#they said a confusion matrix
#3x3 confusion matrix...
#we can have 15 confusion matricies. good way to plot? thatll be  alot of em but a good cisualization
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
for reg in ["L1", "L2", "BOTH"]:
    print(reg)
    for c in [.001, .01, .1, 3]:
        for e in [.001, .01, .1]:
            #now we can print the matricies
            #this may be a lot but its a good idea for our BEST one
            l1 = ls_confusion_matrices[(reg, c,e)]
            lr = lr_confusion_matrices[(reg, c,e)]
            df_cm = pd.DataFrame(l1, index = [i for i in ["Y1", "Y2", "Y3"]],
                      columns = [i for i in ["Y1", "Y2", "Y3"]])
            plt.figure(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
            ax = plt.subplot(1, 2, 1)
            #ax.set_ylim(-0.5, -0.5)
            plt.subplots_adjust(bottom=0.28)
            print(ax.get_ylim(), "LABELES")
            ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
            #plt.gcf().subplots_adjust(top=0.15)
            sn.heatmap(df_cm, annot=True)
            ax = plt.subplot(1,2,2)
            df_cm = pd.DataFrame(lr, index = [i for i in ["Y1", "Y2", "Y3"]],
                      columns = [i for i in ["Y1", "Y2", "Y3"]])
            sn.heatmap(df_cm, annot=True,)
            print(ax.get_ylim(), "LABELES")
            #ax.set_ylim(len(df_cm)-0.5, -0.5)
            ax.set_xlabel('Predicted labels');
            ax.set_ylabel('True labels')
            plt.show()

matplotlib: 3.1.0
L1
(0.0, 1.0) LABELES
(3.0, 0.0) LABELES


<Figure size 1120x480 with 4 Axes>

(0.0, 1.0) LABELES
(3.0, 0.0) LABELES


<Figure size 1120x480 with 4 Axes>

(0.0, 1.0) LABELES
(3.0, 0.0) LABELES


...

<Figure size 1120x480 with 4 Axes>

(0.0, 1.0) LABELES
(3.0, 0.0) LABELES


<Figure size 1120x480 with 4 Axes>

(0.0, 1.0) LABELES
(3.0, 0.0) LABELES


<Figure size 1120x480 with 4 Axes>

In our confusion matrices, you can identify the misidentified classes by their less saturated values where the rows are not correlated with themselves. In most of these matrices, misidentification happened fairly semmetrically, meaning we had an almost equal number of misclassifying a 'fit' as either too small or too large. This indicates that their is not a skew in our model, as it is not favoring one class over the others.

In [31]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA

classif = OneVsRestClassifier(SVC(kernel='linear'))
classif.fit(X_train, y_train)
predict = classif.predict(X_test)
accuracy_score(y_test, predict)

0.684563353061964